#### **Var definitions**
Change it whenever necessary

In [53]:
access_oauth_token = "token <PLACE_HOLDER>" #TODO: delete before commiting the code
repo = "junit-team/junit4"
out = '/Users/lorenapacheco/Concordia/Masters/BugReportsMining/bug-reports/'
repo_name= repo.split('/')[1]
proj = repo_name
base_path = '/Users/lorenapacheco/Concordia/Masters/'
out_commits_dir = base_path + "BugReportsMining/commits/" + repo_name
proj_path = base_path + "open_source_repos_being_studied/" + repo_name

#### **Bug reports mining**
Getting all the Github **issues** from the project that are **closed** and have the **bug**'s label

In [54]:
import os
import json
import requests
import pprint

def get_git_closed_bug_issues(repo, media_type = "application/vnd.github+json"):
    issue_numbers_list = []
    current_page=1
    pagination_end = False
    
    #iteration throught the pagination
    while not pagination_end:
        issues_json = requests.get(
            "https://api.github.com/repos/"+repo+"/issues?state=closed&labels=bug&page="+str(current_page),
            headers={
                'Accept': media_type,
                'Authorization': access_oauth_token
            }
        ).json()
        if len(issues_json) > 0:
            for issue in issues_json:
                issue_numbers_list.append(issue["number"])
            current_page = current_page + 1
        else:
            pagination_end = True
            
    return issue_numbers_list


issue_numbers_list = get_git_closed_bug_issues(repo)
print(issue_numbers_list)
print("\nNumber of issues: " + str(len(issue_numbers_list)))

[1616, 1508, 1507, 1462, 1370, 1365, 1363, 1334, 1320, 1290, 1238, 1236, 1224, 1213, 1192, 1189, 1178, 1083, 1073, 1014, 896, 894, 449, 441, 379, 359, 350, 338, 333, 332, 311, 309, 303, 296, 295, 292, 289, 262, 260, 258, 257, 248, 242, 235, 227, 226, 225, 220, 219, 218, 209, 208, 192, 191, 188, 187, 177, 165, 150, 134, 131, 125, 121, 120, 119, 105, 104, 98, 96, 94, 89, 88, 84, 82, 81, 76, 74, 73, 62, 61, 55, 49, 48, 42, 39, 38, 35, 33, 28, 23]

Number of issues: 90


#### **Bug reports content extraction**
Extraction the content of each of these bug reports

In [55]:
output_folder = out + repo_name+ '/'

def git_request_br_json(issue_n, repo, media_type = 'application/vnd.github.v3+json'):
    retrieved_json = requests.get(
        'https://api.github.com/repos/'+repo+'/issues/'+str(issue_n),
        headers={
            'Accept': media_type,
            'Authorization': access_oauth_token
        }
    ).json()
    #pprint.pprint(retrieved_json)
    
    if 'comments_url' not in retrieved_json:
        return
    
    comment_url = retrieved_json['comments_url']
    retrieved_comments = requests.get(
        comment_url,
        headers={
            'Accept': media_type,
            'Authorization': access_oauth_token
        }
    ).json()
    retrieved_json['comments_content'] = retrieved_comments
    return retrieved_json

def dict_to_json_file(file, dic, folder=output_folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()


for issue_number in issue_numbers_list:
    try:
        retrieved_json = git_request_br_json(issue_number, repo)
    except ConnectionError:
        print('ConnectionError', repo, issue_number)
        break

    if not retrieved_json:
        print('Json not retrieved, something might be wrong')
        continue

    dict_to_json_file(repo_name+'-'+str(issue_number), retrieved_json)
print ("Extraction completed")

Extraction completed


#### **Filtering for logs and stack traces**
Filtering only the bug reports with **logs** and **stack traces**

In [56]:
import re
import glob
from collections import defaultdict

# global dict of a dict for storing the log
bug_report_log = defaultdict(dict)
bug_report_stack_trace = defaultdict(dict)

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(output_folder, file+'.json'), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def find_regex(regex, var,file_name, text_content):
    results = re.finditer(regex, text_content)
    if results:
        for log in results:
            bug_name = file_name.split('-')[1]
            var[proj][bug_name] = log.group() 
            
            
def find_logs_and_stack_traces (file_name, text_content):
    stack_trace_regex = r'(?m)^.*?Exception.*(?:\n+^\s*at .*)+'
    logs_regex = r'(ERROR|INFO|WARN|DEBUG|FATAL)\s+(?P<class>\w+(\.\w+)*)'
    find_regex(logs_regex, bug_report_log,file_name, text_content)
    find_regex(stack_trace_regex, bug_report_stack_trace,file_name, text_content)
    

# for every .json in bug-reports folder
for file in glob.glob(output_folder +'/*.json'):
    file_name = os.path.basename(file).replace('.json', '')
    proj = file_name.replace('-'+file_name.split('-')[-1], '').lower()
    bug_report = json_file_to_dict(file.replace('.json', ''))
    if not bug_report:
        continue
    
    title = bug_report['title'] if bug_report['title'] else ""
    description = bug_report['body'] if bug_report['body'] else ""
    text_content = title + '\n' + description + '\n' + '\n'.join([comment['body'] for comment in bug_report['comments_content'] if 'body' in comment and comment['body']])
    
    find_logs_and_stack_traces(file_name, text_content)

print("Bug reports with logs:" + str(len(bug_report_log[proj])))
print("Bug reports with stack traces:" + str(len(bug_report_stack_trace[proj])))

Bug reports with logs:0
Bug reports with stack traces:12


#### **Commits search**
Searching for the respective commits

In [57]:
from urllib.parse import quote

bug_reports_with_logs_commits = defaultdict(dict)
bug_reports_with_stack_traces_commits = defaultdict(dict)
cont = 0

def get_bug_report_commit(bug, proj_path):
    os.chdir(proj_path)
    log_grep_command = "git log --grep="+ bug+" --pretty=format:\"%H\" >> commit_output"
    os.system(log_grep_command)
    lines = []
    with open("commit_output", 'r') as fp:
        lines =fp.readlines()
        fp.close()
    os.system("rm commit_output")
    if (lines):
        commit_hash = lines[0]
        return(commit_hash)
    return ""

for bug in bug_report_log[proj].keys():
    commit = get_bug_report_commit(bug, proj_path)
    if (commit != ""):
        content = {
            "log": bug_report_log[proj][bug],
            "commit": commit.strip()
        }
        bug_reports_with_logs_commits[bug] = content
        cont = cont + 1

for bug in bug_report_stack_trace[proj].keys():
    commit = get_bug_report_commit(bug, proj_path)
    if (commit != ""):
        content = {
            "stack_trace": bug_report_stack_trace[proj][bug],
            "commit": commit.strip()
        }
        bug_reports_with_stack_traces_commits[bug] = content
        cont = cont + 1

if bug_reports_with_logs_commits:
    dict_to_json_file("bug_reports_with_logs_commits", bug_reports_with_logs_commits, out_commits_dir)
if bug_reports_with_stack_traces_commits:
    dict_to_json_file("bug_reports_with_stack_traces_commits", bug_reports_with_stack_traces_commits, out_commits_dir)

print("Extraction complete")
print("Number obtained: " + str(cont))

Extraction complete
Number obtained: 11
